# Make a copy of this notebook! 

# Intro to Colab

**60 second crash course in Colab notebooks**

A notebook is a list of cells. Cells contain either **explanatory text** or **executable code** and its output. This is a **text cell**. You can double-click to edit this cell.

Once the toolbar button indicates CONNECTED, click in the cell to select it and execute the contents in the following ways:

* Click the **Play icon** in the left gutter of the cell; or
* Type **Cmd/Ctrl + Enter** to run the cell in place.

Good to know
* **Hashtags (#)** are Python comments (they're ignored during code execution)
* Use **Cmd/Ctrl + / ** to comment out a line of code (helpful during debugging)
* When you execute a code block, anything within that code block can be referenced elsewhere in the notebook

In [ ]:
# Printing to screen
print("I'm a code block")

# Defining variables
a = 2
b = 5
c = a + b
print(f"a equals {a}")
print(f"b equals {b}")
print(f"a plus b equals {c}")

# Proper indentation is essential in Python
for x in range(1,6):
  print(x)

# Alligator 2.0 Notebook

This notebook contains the steps necessary to set up the Google My Business (GMB) [Insights](https://developers.google.com/my-business/reference/rest/v4/Metric) extraction and aggregation tool known as [Alligator 2.0](https://github.com/google/alligator2), as well as the visualization of the extracted insights in a Google Data Studio dashboard.

## Solution Details

### Overview


Alligator 2.0 (Alligator for short) is a Python-based solution that aggregates *Insights* data from the GMB API and stores it into Google Cloud Platform, precisely in BigQuery. *Insights* data provides details around how users interact with GMB listings via Google Maps, such as the number of queries for a location, locations of where people searched for directions, number of website clicks, calls, and reviews.

The solution provides a cross-account look at the GMB data, instead of a per-location view. In addition, the use of BigQuery to aggregate and store this data allows trends longer than the range accessible through the GMB API to be captured.

Along with gathering stats, the Natural Language API from Cloud has been used to provide sentiment analysis, as well as entity detection for supported languages.

### Architecture

Alligator is a process built in Python that pulls data from the GMB API and gathers it into a BigQuery instance.

The process uses the publicly available GMB API, along with the Big Query API to access, download and aggregate the data. Access is gained using standard OAuth tokens, which are stored in the runtime environment executing the Python process.

Once aggregated, tables are created in BigQuery, and the reviews are processed using the NLP API on Cloud to get their sentiment information.




## GMB Account Prerequisites


*   All locations must roll up to a `Location Group` (formerly known as a `Business Account`). If this is not the case, please create a single Location Group and add all locations to it. Click [here](https://support.google.com/business/answer/6085339?ref_topic=6085325) for more information.<br/>Multiple location groups are supported and can be queried accordingly (refer to the [BigQuery Views section](#bigquery-views) below).
*   The user accessing the GMB UI must have *Admin* privileges set for the Location Group(s).



## GCP Project Setup

To get started, you first need to create or select an existing Google Cloud Platform Project that is allowed to access the GMB API. For more information on getting access, refer to [this guide](https://developers.google.com/my-business/content/prereqs#request-access).

Once your project is created and approved, the GMB API will automatically be enabled for use within the project (if not, enable it using this [link](https://console.developers.google.com/start/api?id=mybusiness.googleapis.com&credential=client_key)). Additionally, please enable the following APIs:

*   [Cloud Natural Language API](https://console.developers.google.com/start/api?id=language.googleapis.com&credential=client_key)
*   [BigQuery API](https://console.developers.google.com/start/api?id=bigquery&credential=client_key)

Make sure that the user with access to the GMB UI/API also has permission to access the other APIs as well.

### Authentication

The APIs are authenticated via OAuth, which means that you authenticate as a user as opposed to as an application. Your user permissions control what you have access to. If you don't have access in the UI you cannot access via the API.

The client secrets give your application permission to access the API using your user credentials.

1. Go to the [API & Services > Credentials](https://console.cloud.google.com/apis/credentials) page on Google Cloud and select the project for which you enabled the APIs earlier.
1. Create an OAuth 2.0 Client ID of the type *Desktop App*.
1. Download the client secrets json file and save it as `client_secrets.json`.

  > ![Download JSON](https://cloud.google.com/bigquery/images/download-json.png)


In [ ]:
#@markdown Authenticate your user for this colab
from google.colab import auth
auth.authenticate_user()

## Installation Guide

In [ ]:
#@markdown Set a global flag representing your GCP Project ID
PROJECT_ID = '<your-project-id-here>'  # @param

In [ ]:
#@markdown Clone the alligator2 GitHub repository and cd into it

!echo "Restoring working directory to root..."
%cd /content
!rm -rf alligator2 && git clone https://github.com/google/alligator2.git
!echo "Changing working directory to alligator2..."
%cd alligator2

In [ ]:
#@markdown Copy the client_secrets.json file into the alligator2 directory

from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('Uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
#@markdown Install the application's required packages

!pip3 install --upgrade --requirement requirements.txt

In [ ]:
#@markdown Download the GMB API discovery document
#@markdown <br/>*Refer to this*
#@markdown [*link*](https://developers.google.com/my-business/samples/#discovery_document)
#@markdown *for the latest available discovery document as the one used here may have been outdated*

!wget -O gmb_discovery.json https://developers.google.com/my-business/samples/mybusiness_google_rest_v4p7.json

Now you are all set; execute the cell below to run the application and start extracting and aggregating GMB data!

Refer to the [CLI Usage](https://github.com/google/alligator2#usage-of-the-cli) section of the GitHub documentation for more information on the different flags you can use.

In [ ]:
!python3 main.py --project_id=$PROJECT_ID

Depending on the amount of GMB source data you have across your locations and location groups, the frequency of analysis may differ.
<br/>Usually, a weekly workflow suffices for capturing enough insights. Refer to the [Maintenance Guide](#scrollTo=LbIu0fWANAw5) section below for more information.

You can also fine-tune the constants referenced in [api.py](https://github.com/google/alligator2/blob/master/api.py) to fit your data volume and indiviual use cases. The predefined values are optimized for the extraction of significantly large volumes of data without running into [operational](https://cloud.google.com/bigquery/quotas#streaming_inserts) / transactional limits.

```python
DATASET_ID = "alligator"

# max lookback window for insights metrics in days
INSIGHTS_DAYS_BACK = 540 

# calls metrics are aggregated in GMB, this value supports a weekly workflow
CALLS_DAYS_BACK = 7 

# directions metrics are limited to predefined values in GMB, this value supports a weekly workflow
DIRECTIONS_NUM_DAYS = "SEVEN" 

# max number of rows of data to retrieve from BQ. Used when processing sentiments for existing reviews
BQ_JOBS_QUERY_MAXRESULTS_PER_PAGE = 1000 

# max batch size for BQ streaming inserts. Do not change this value as you might run into API limits
BQ_TABLEDATA_INSERTALL_BATCHSIZE = 50
```



Once the script is run, your BigQuery dataset will be populated with several tables depending on the execution flags used.<br/>
Execute the next cell to view your dataset's contents (make sure you change the dataset name if you are not using the default 'alligator').

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  alligator.INFORMATION_SCHEMA.TABLES;

Here is a description of each table and the extraction methods used: 

*   **accounts**: Provides information on the accounts (e.g., location groups) associated to the user. It is omitted if running for a specific account using the `--account_id` switch.
*   **locations**: Provides information about locations associated to the user through the GMB accounts. It is omitted if running for a specific location using the `--location_id` switch.
*   **insights**: Executes the [reportInsights](https://developers.google.com/my-business/reference/rest/v4/accounts.locations/reportInsights) method and gets all the information under Location Metrics, aggregated by day, for the last 540 days (with 5 days delay) for all associated locations. This range can be modified in the code.
*   **directions**: Executes the [reportInsights](https://developers.google.com/my-business/reference/rest/v4/accounts.locations/reportInsights) method and gets all the information under Driving Direction Metrics, for the last 7 days for all associated locations. This range can be modified to one of the allowed ones (7, 30 or 90 days).
*   **hourly_calls**: Executes the [reportInsights](https://developers.google.com/my-business/reference/rest/v4/accounts.locations/reportInsights) method and gets all the phone actions aggregated by hour of the day, for the last 7 days (with 5 days delay) for all associated locations. This range can be modified in the code, but consistency of the data will depend on the extraction schedule and the range complementariness.
*   **reviews**: Provides all the reviews available in the API for all associated locations.
*   **sentiments**: Provides the sentiment extracted for all the reviews present in the reviews table at execution time.


## Maintenance Guide

Data extracted by Alligator needs to be maintained in order to be consistent, up to date and GDPR compliant.

On the compliance side, any user can delete a review in GMB, and the data owner is responsible for deleting all copies of it in their systems. Unfortunately the GMB API does not provide a means to get information on deleted reviews. The workaround for this is to extract all reviews periodically, and delete old copies from the data lake.

To ensure consistency, and as the extraction process could duplicate information, all the tables are partitioned by day of extraction to facilitate deletion of old data.

On the same note, the `directions` and `hourly_calls` are different from other tables in that their information is aggregated (`hourly_calls`), or limited to predefined ranges (`directions`) in the API. For that, it is recommended to avoid deleting these two tables periodically, in case you want to extend the lookback window farther than as provided by the API.

### Weekly Workflow

Alligator is prepared to use this workflow, and we propose the following approach to handle extractions and deletions.

*   Day 01: the extraction is run completely.
*   Day 02 to 07: nothing.
*   Day 08: prior to running the extraction completely, manually delete all data with `_PARTITIONTIME < TODAY` from all tables, except for *directions* and *hourly_calls*.
*   Day 08 to 14: nothing.
*   Day 15: prior to running the extraction completely, manually delete all data with `_PARTITIONTIME < TODAY` from all tables, except for *directions* and *hourly_calls*.
<br/>...


### Daily Workflow

In case there is a need to get information as soon as possible, there is an alternative approach, but it might require a bit more maintenance.

*   Day 01: the extraction is run completely.
*   Day 02: the extraction is run with the `--no-directions` and `--no-hourly-calls` flag. Before doing that, all data with `_PARTITIONTIME < TODAY` should be manually deleted from all tables, except for *directions* and *hourly_calls*.
*   Day 03: (same as Day 02)
*   Day 04: (same as Day 02)
*   Day 05: (same as Day 02)
*   Day 06: (same as Day 02)
*   Day 07: (same as Day 02)
*   Day 08: prior to running the extraction completely, manually delete all data with `_PARTITIONTIME < TODAY` from all tables, except for *directions* and *hourly_calls*.
*   Day 09: (same as Day 02)
*   Day 10: (same as Day 02)
*   Day 11: (same as Day 02)
*   Day 12: (same as Day 02)
*   Day 13: (same as Day 02)
*   Day 14: (same as Day 02)
*   Day 15: prior to running the extraction completely, manually delete all data with `_PARTITIONTIME < TODAY` from all tables, except for *directions* and *hourly_calls*.
<br/>...


## Reporting Guide

Now that the data is available in BigQuery we can build the Data Studio Dashboard. First, let's try to visualize some of the data you have extracted so far. As an example, the code below will aggregate the star rating of all your reviews (across all your locations):

In [ ]:
%%bigquery reviews_agg_rating --project $PROJECT_ID

SELECT
  r.starRating AS rating,
  count(1) AS count_rating
FROM 
  alligator.reviews AS r
GROUP BY r.starRating
ORDER BY (
  CASE r.starRating
    WHEN "ONE" then 1
    WHEN "TWO" then 2
    WHEN "THREE" then 3
    WHEN "FOUR" then 4
    WHEN "FIVE" then 5
  END
);

Now let's plot the data using [matplotlib](https://matplotlib.org/).

In [ ]:
%matplotlib inline
reviews_agg_rating.plot(kind="bar", x="rating", y="count_rating");

Let's keep going and identify the top 10 locations based on review rating:

In [ ]:
%%bigquery --project $PROJECT_ID

WITH location_reviews AS (
  SELECT 
    l.locationName,
    (CASE
      WHEN r.starRating = "ONE" THEN 1
      WHEN r.starRating = "TWO" THEN 2
      WHEN r.starRating = "THREE" THEN 3
      WHEN r.starRating = "FOUR" THEN 4
      WHEN r.starRating = "FIVE" THEN 5
      ELSE NULL
    END) AS numRating
  FROM 
    alligator.reviews AS r
  JOIN 
    alligator.locations AS l
      ON l.name = REGEXP_EXTRACT(r.name, '(.*)/reviews/')
)
SELECT
  locationName,
  AVG(numRating) AS avgRating
FROM 
  location_reviews
WHERE numRating IS NOT NULL
GROUP BY locationName
ORDER BY avgRating DESC
LIMIT 10;

Now that you have an idea of the different visualizations possible with this data, let's build the Data Studio Dashboard. You can use [this](https://datastudio.google.com/c/reporting/d86ca1bb-4255-4089-9adf-b99148bc5a94) dashboard template and copy it as-is, but first you will need to prepare the data sources beforehand. 

For this, you have two options:

*   Create views within your GCP project's BigQuery dataset to supply the data to the dashboard; or
*   Use Data Studio's *Custom Query* interface to directly input the views' SQL queries.

It is recommended that you follow the first approach (views in BigQuery) for maintainability and performance reasons.

*   To create a view in BigQuery, you can follow the steps described [here](https://cloud.google.com/bigquery/docs/views). You will need to create six views, and you can find sample SQL files in the [sql folder](https://github.com/google/alligator2/tree/master/sql) of the project (make sure to modify the Project ID and BigQuery dataset if you are not using the default).
>Once the views are available, you need to link them to Data Studio by creating Data Sources as described [here](https://support.google.com/datastudio/answer/6370296?hl=en), and selecting the *My Projects or Shared Projects* options.

*   If you prefer to go the *Custom Query* route, you can use the same [instructions](https://support.google.com/datastudio/answer/6370296?hl=en) to create the Data Sources but selecting the *Custom Query* option. You can also use the [sql samples](https://github.com/google/alligator2/tree/master/sql) as the custom queries on each Data Source.

After the Data Sources are created, you can create a copy of the template and replace the sample data sources with the ones associated with your project.

### Geographical Visualization for Driving Directions

> *At the time of writing, Data Studio's built in _Geo_ and _Google Maps_ chart types did not support custom map overlays.*

For example, say we want to plot both an existing location as a **pointer** on the map, along with a **polygon** representing all source postal codes from which driving directions originated.

Since this is not currently possible with Data Studio, [BigQuery Geo Viz](https://bigquerygeoviz.appspot.com/) and/or [geojson.io](https://geojson.io) (among others) can be used instead.

Follow the instructions below to plot this information for a given city and postal code pair using geojson.io. You could also materialize the SQL below as a view in BigQuery and use BigQuery Geo Viz for a more aggregated view (e.g. city-wide or even regional) of the data.

In [ ]:
#@markdown Enter the desired plotting params
#@markdown <br/>*param_zipcode* and *param_address* are optional, 
#@markdown though it is highly recommended that you enter a postal code to view the effect of having multiple locations within the same area:
param_city = '<city>'  # @param
param_zipcode = '<zipcode>'  # @param
param_address = ''  # @param

params = {
    'param_city': param_city,
    'param_zipcode': param_zipcode,
    'param_address': param_address
}

In [ ]:
#@markdown Fetch data from BigQuery and store it in a Pandas DataFrame named 'geo_data'.
%%bigquery geo_data --project $PROJECT_ID --params $params

WITH temp_data AS (
  SELECT
    regionCode AS countryCode,
    locality AS city,
    postalCode AS locationZipCode,
    locationName AS locationName,
    addressLines AS locationAddress,
    label AS sourceZipCode,
    latitude,
    longitude
  FROM 
    alligator.formatted_directions
  WHERE SAFE_CAST(label AS NUMERIC) IS NOT NULL
  ORDER BY city, locationName, locationAddress, locationZipCode, sourceZipCode
), temp_aggregated_data AS (
  SELECT distinct
    regionCode AS countryCode,
    locality AS city,
    postalCode AS destinationZipCode,
    label AS sourceZipCode,
    latitude,
    longitude
  FROM 
    alligator.formatted_directions
  WHERE SAFE_CAST(label AS NUMERIC) IS NOT NULL
  ORDER BY city, destinationZipCode, sourceZipCode
), temp_geo_data AS (
  SELECT
    d.countryCode,
    d.city,
    d.locationZipCode,
    d.locationName,
    d.locationAddress,
    ARRAY_AGG(d.sourceZipCode ORDER BY d.sourceZipCode) AS sourceZipCodes,
    ARRAY_AGG(ST_GeogPoint(d.longitude, d.latitude) ORDER BY d.sourceZipCode) AS sourceGeoPoints
  FROM 
    temp_data AS d
  GROUP BY countryCode, city, locationZipCode, locationName, locationAddress
  ORDER BY countryCode, city, locationZipCode, locationName, locationAddress
), temp_aggregated_geo_data AS (
  SELECT
    a.countryCode,
    a.city,
    a.destinationZipCode,
    ARRAY_AGG(a.sourceZipCode ORDER BY a.sourceZipCode) AS sourceZipCodes,
    ARRAY_AGG(ST_GeogPoint(a.longitude, a.latitude) ORDER BY a.sourceZipCode) AS sourceGeoPoints
  FROM 
    temp_aggregated_data AS a
  GROUP BY countryCode, city, destinationZipCode
  ORDER BY countryCode, city, destinationZipCode
), directions_geo AS (
  SELECT 
    ag.countryCode,
    ag.city,
    ag.destinationZipCode,
    ag.sourceZipCodes,
    CASE 
      WHEN 
        ST_NUMPOINTS(ST_MAKELINE(ag.sourceGeoPoints)) >= 3 
      THEN 
        ST_GEOGFROMGEOJSON(REPLACE(REPLACE(REPLACE(ST_ASGEOJSON(ST_MAKELINE(ag.sourceGeoPoints)), 'LineString', 'Polygon'), ': [ [', ':[[['), '] ] } ', ']]]}'), make_valid => true)
      ELSE 
        ST_MAKELINE(ag.sourceGeoPoints)
    END AS sourceZipCodesGeoPolygon,
    g.locationName,
    g.locationAddress,
    g.sourceZipCodes AS locationSourceZipCodes,
    CASE 
      WHEN 
        ST_NUMPOINTS(ST_MAKELINE(g.sourceGeoPoints)) >= 3 
      THEN 
        ST_GEOGFROMGEOJSON(REPLACE(REPLACE(REPLACE(ST_ASGEOJSON(ST_MAKELINE(g.sourceGeoPoints)), 'LineString', 'Polygon'), ': [ [', ':[[['), '] ] } ', ']]]}'), make_valid => true)
      ELSE 
        ST_MAKELINE(g.sourceGeoPoints)
    END AS locationSourceZipCodesGeoPolygon
  FROM 
    temp_aggregated_geo_data AS ag
  JOIN temp_geo_data AS g
    ON g.locationZipCode = ag.destinationZipCode
  ORDER BY ag.countryCode, ag.city, ag.destinationZipCode, g.locationAddress
)
# The aformentioned temp tables can be stored as a BigQuery view named
# directions_geo and queried below directly as alligator.directions_geo
SELECT
  city,
  destinationZipCode,
  locationName,
  locationAddress,
  ST_ASGEOJSON(locationSourceZipCodesGeoPolygon) AS geojson
FROM
  directions_geo
WHERE
  city = @param_city
  AND destinationZipCode = (
    CASE 
      WHEN 
        @param_zipcode = '' 
      THEN
        destinationZipCode
      ELSE
        @param_zipcode
    END
  )
  AND locationAddress = (
    CASE 
      WHEN 
        @param_address = '' 
      THEN
        locationAddress
      ELSE
        @param_address
    END
  )
;

In [ ]:
#@markdown Output the DataFrame:
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None)

geo_data

In [ ]:
#@markdown We need to fetch a latitude, longitude pair for every location within the provided postal code.
#@markdown <br/>Multiple locations will be visualized using different colors.
#@markdown <br/><br/>Execute this cell to install all required packages.
!pip3 install -U geocoder geojson

In [ ]:
#@markdown Execute this cell to create a [GeoJSON specification](https://geojson.org/) string.
import geocoder
import json
import random

from geojson import FeatureCollection, Feature, Point


def geocode(city, zipcode, address):
  g = geocoder.osm(f"{address} {zipcode} {city}", components="country:DE")
  return g.lng, g.lat


def geodata_feature_point(row, color):
  city = row.city
  zipcode = row.destinationZipCode
  name = row.locationName
  address = row.locationAddress
  
  lng, lat = geocode(city, zipcode, address)
  point = Point((lng, lat))

  feature = Feature(geometry=point, properties={
      'name': f'Location Pointer',
      'popupContent': f'{name}<br>{address}<br>{zipcode} {city}',
      'color': color
  })

  return feature


def geodata_feature_polygon(row, color):
  city = row.city
  zipcode = row.destinationZipCode
  name = row.locationName
  address = row.locationAddress

  j = json.loads(row.geojson)
  feature = Feature(geometry=j, properties={
      'name': f'Driving Directions Polygon',
      'popupContent': f'Driving directions for:<br/>{name}<br>{address}<br>{zipcode} {city}',
      'color': color
  })
  
  return feature


def gen_colors(number_of_colors):
  colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
               for i in range(number_of_colors)]
  
  return colors


def geodata_features():
  features = []
  points = []
  polygons = []

  colors = gen_colors(len(geo_data))

  for index, row in geo_data.iterrows():
    color = colors[index]
    feature_point = geodata_feature_point(row, color)
    points.append(feature_point)

    feature_polygon = geodata_feature_polygon(row, color)
    polygons.append(feature_polygon)

  # Add polygons first to ensure that points are plotted above them
  features.extend(polygons)
  features.extend(points)

  return features


address = None
if param_address != '':
  address = param_address

lng, lat = geocode(param_city, param_zipcode, address)
feature_collection = FeatureCollection(geodata_features())

In [ ]:
#@markdown Now execute this cell to visualize the data.
from IPython.display import HTML

# Single curly braces are reserved for "replacement fields", escape curly braces in literal text by doubling them.
display(HTML(f'''
<link rel="stylesheet" href="https://unpkg.com/leaflet@1.7.1/dist/leaflet.css" integrity="sha512-xodZBNTC5n17Xt2atTPuE1HxjVMSvLVW9ocqUKLsCC5CXdbqCmblAshOMAS6/keqq/sMZMZ19scR4PsZChSR7A==" crossorigin=""/>
<script src="https://unpkg.com/leaflet@1.7.1/dist/leaflet.js" integrity="sha512-XQoYMqMTK8LvdxXYG3nZ448hOEQiglfqkJs1NOQV44cWnUrBc8PkAOcXy20w0vlaXaVUearIOBhiXZ5V3ynxwA==" crossorigin=""></script>

<div id="mapid" style="width: 600px; height: 400px;"></div>
<script>
  function onEachFeature(feature, layer) {{
    if (feature.properties && feature.properties.popupContent) {{
      layer.bindPopup(feature.properties.popupContent);
    }}
  }}

  function styles(feature) {{
    if (feature.properties && feature.properties.color) {{
      return {{
        color: feature.properties.color
      }};
    }}
  }}

  function pointToLayer(feature, latlng) {{
    var options = geojsonMarkerOptions;

    if (feature.properties && feature.properties.color) {{
       options.fillColor = feature.properties.color;
    }}    
    return L.circleMarker(latlng, options);
  }}

  var geojsonMarkerOptions = {{
    radius: 8,
    fillColor: "#ff7800",
    color: "#000",
    weight: 1,
    opacity: 1,
    fillOpacity: 0.8
  }};

	var map = L.map('mapid').setView([{lat}, {lng}], 12);

	L.tileLayer('https://api.mapbox.com/styles/v1/{{id}}/tiles/{{z}}/{{x}}/{{y}}?access_token=pk.eyJ1IjoibWFwYm94IiwiYSI6ImNpejY4NXVycTA2emYycXBndHRqcmZ3N3gifQ.rJcFIG214AriISLbB6B5aw', {{
		maxZoom: 18,
		attribution: 'Map data &copy; <a href="https://www.openstreetmap.org/">OpenStreetMap</a> contributors, ' +
			'<a href="https://creativecommons.org/licenses/by-sa/2.0/">CC-BY-SA</a>, ' +
			'Imagery © <a href="https://www.mapbox.com/">Mapbox</a>',
		id: 'mapbox/streets-v11',
		tileSize: 512,
		zoomOffset: -1
	}}).addTo(map);

  L.geoJSON({feature_collection}, {{
      onEachFeature: onEachFeature,
      style: styles,
      pointToLayer: pointToLayer
  }}).addTo(map);
</script>
'''))